# Maximum common substructure (MCS)

MolecularGraph.js implements several type of MCS for cheminformatics applications.

- Node induced (MCIS) / Edge induced (MCES)
- Connected / disconnected
- Topological constraint (tdMCS)
- Graph based local similarity (GLS)


In [1]:
import Pkg
Pkg.activate("..")
using MolecularGraph
using MolecularGraph.MolecularGraphModel


#  Download test data from PubChem

pubchemsdf("6047", "Levodopa")
pubchemsdf("74217", "3-Aminocoumarin")
pubchemsdf("6437877", "Cefditoren Pivoxil")
pubchemsdf("5481173", "Ceftazidime")

┌ Info: Recompiling stale cache file /Users/smatsuoka/.julia/compiled/v1.1/MolecularGraph/6vz1i.ji for MolecularGraph [6c89ec66-9cd8-5372-9f91-fabc50dd27fd]
└ @ Base loading.jl:1184


loading: funcgroup.yaml
loading: ring.yaml
loading: biomolecule.yaml
file: Levodopa.mol already exists
file: 3-Aminocoumarin.mol already exists
file: Cefditoren Pivoxil.mol already exists
file: Ceftazidime.mol already exists


## MCIS / MCES


In [2]:
ldopa = sdftomol(joinpath(pubchem_dir, "Levodopa.mol"))
ldopa = graphmol(makehydrogensimplicit(ldopa))
display("image/svg+xml",  drawsvg(ldopa, 180, 180))

aminocoumarin = sdftomol(joinpath(pubchem_dir, "3-Aminocoumarin.mol"))
aminocoumarin = graphmol(makehydrogensimplicit(aminocoumarin))
display("image/svg+xml",  drawsvg(aminocoumarin, 150, 150))

In [3]:
mcis = mcismol(ldopa, aminocoumarin)

subg1 = nodesubgraph(ldopa, Set(keys(mcis.mapping)))
display("image/svg+xml", drawsvg(subg1, 180, 180))

subg2 = nodesubgraph(aminocoumarin, Set(values(mcis.mapping)))
display("image/svg+xml",  drawsvg(subg2, 150, 150))

println("MCIS size: $(length(mcis.mapping))")

MCIS size: 9


In [4]:
mces = mcesmol(ldopa, aminocoumarin)

subg1 = edgesubgraph(ldopa, Set(keys(mces.mapping)))
display("image/svg+xml", drawsvg(subg1, 180, 180))

subg2 = edgesubgraph(aminocoumarin, Set(values(mces.mapping)))
display("image/svg+xml",  drawsvg(subg2, 150, 150))

println("MCES size: $(length(mces.mapping))")

MCES size: 8


In [6]:
mcis = mcismol(ldopa, aminocoumarin, connected=true)

subg1 = nodesubgraph(ldopa, Set(keys(mcis.mapping)))
display("image/svg+xml", drawsvg(subg1, 180, 180))

subg2 = nodesubgraph(aminocoumarin, Set(values(mcis.mapping)))
display("image/svg+xml",  drawsvg(subg2, 150, 150))

println("MCIS size: $(length(mcis.mapping))")

MCIS size: 9


In [8]:
mces = mcesmol(ldopa, aminocoumarin, connected=true)

subg1 = edgesubgraph(ldopa, Set(keys(mces.mapping)))
display("image/svg+xml", drawsvg(subg1, 180, 180))

subg2 = edgesubgraph(aminocoumarin, Set(values(mces.mapping)))
display("image/svg+xml",  drawsvg(subg2, 150, 150))

println("MCES size: $(length(mces.mapping))")

MCES size: 8


In [3]:
mol1 = sdftomol(joinpath(pubchem_dir, "Cefditoren Pivoxil.mol"))
mol1 = graphmol(makehydrogensimplicit(mol1))
mol_svg = drawsvg(mol1, 300, 300)
display("image/svg+xml",  mol_svg)

In [4]:
mol2 = sdftomol(joinpath(pubchem_dir, "Ceftazidime.mol"))
mol2 = graphmol(makehydrogensimplicit(mol2))
mol_svg = drawsvg(mol2, 300, 300)
display("image/svg+xml",  mol_svg)

## MCS calculation

- `mcesmol`

In [5]:
Profile.clear()
@profile mcesmol(mol1, mol2, topological=true, timeout=10)
Profile.print(mincount=10)

1257 ./task.jl:259; (::getfield(IJulia, Symbol("##15#1...
 1257 ...fjEtl/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  1257 ./essentials.jl:741; invokelatest
   1257 ./essentials.jl:742; #invokelatest#1
    1257 ...c/execute_request.jl:67; execute_request(::ZMQ.Socket, ::...
     1256 .../SoftGlobalScope.jl:218; softscope_include_string(::Modu...
      1256 ./boot.jl:328; eval
       371 ...mpiler/typeinfer.jl:613; typeinf_ext(::Core.MethodInsta...
        371 ...mpiler/typeinfer.jl:576; typeinf_ext(::Core.MethodInsta...
         371 ...mpiler/typeinfer.jl:14; typeinf(::Core.Compiler.Infere...
          371 ...interpretation.jl:1191; typeinf_nocycle(::Core.Compi...
           365 ...interpretation.jl:1135; typeinf_local(::Core.Compil...
            365 ...interpretation.jl:890; abstract_eval(::Any, ::Arra...
             365 ...interpretation.jl:805; abstract_eval_call(::Array{...
              365 ...nterpretation.jl:776; abstract_call(::Any, ::Arr...
               365 ...nterpretati

                                    +35 29 ...ion.jl:890; abstract_ev...
                                    +36 29 ...ion.jl:805; abstract_ev...
                                    +37 29 ...ion.jl:776; abstract_ca...
                                    +38 24 ...ion.jl:85; abstract_ca...
                                    +39 24 ...ion.jl:345; abstract_ca...
                                    +40 24 ...fer.jl:497; typeinf_edg...
                                    +41 21 ...nfer.jl:14; typeinf(::Co...
                                    +42 21 ...ion.jl:1191; typeinf_no...
                                    +43 20 ...ion.jl:1135; typeinf_lo...
                                    +44 20 ...ion.jl:890; abstract_ev...
                                    +45 20 ...ion.jl:805; abstract_ev...
                                    +46 20 ...ion.jl:776; abstract_ca...
                                    +47 17 ...ion.jl:85; abstract_ca...
                                    +48 17 ...ion.jl

                                    +19 12 ...ion.jl:567; abstract_ca...
                                    +20 12 ...ion.jl:519; abstract_ap...
                                    +21 12 ...ion.jl:776; abstract_ca...
                                    +22 12 ...tion.jl:85; abstract_cal...
                                    +23 12 ...ion.jl:345; abstract_ca...
                                    +24 12 ...fer.jl:497; typeinf_edg...
                                    +19 16 ...ion.jl:776; abstract_ca...
                                    +20 16 ...tion.jl:85; abstract_cal...
                                    +21 16 ...ion.jl:345; abstract_ca...
                                    +22 16 ...fer.jl:497; typeinf_edg...
                                    +23 15 ...nfer.jl:14; typeinf(::Co...
                                    +24 15 ...ion.jl:1191; typeinf_no...
                                    +25 15 ...ion.jl:1135; typeinf_lo...
                                    +26 15 ...io

         57  ...piler/typeinfer.jl:613; typeinf_ext(::Core.MethodInst...
          57 ...piler/typeinfer.jl:576; typeinf_ext(::Core.MethodInst...
           55 ...piler/typeinfer.jl:14; typeinf(::Core.Compiler.Infer...
            55 ...interpretation.jl:1191; typeinf_nocycle(::Core.Comp...
             49 ...interpretation.jl:1135; typeinf_local(::Core.Compil...
              49 ...interpretation.jl:890; abstract_eval(::Any, ::Arra...
               49 ...nterpretation.jl:805; abstract_eval_call(::Array...
                49 ...nterpretation.jl:776; abstract_call(::Any, ::Ar...
                 49 ...nterpretation.jl:85; abstract_call_gf_by_type(...
                  49 ...terpretation.jl:345; abstract_call_method(::M...
                   40 ...er/typeinfer.jl:497; typeinf_edge(::Method, :...
                    37 ...er/typeinfer.jl:14; typeinf(::Core.Compiler....
                     37 ...rpretation.jl:1191; typeinf_nocycle(::Core...
                      26 ...rpretation.jl:1121;

In [9]:
mcs = mcesmol(mol1, mol2, topological=true, timeout=10)

MolecularGraph.MolecularGraphModel.MCSResult(Dict(30=>27,1=>1,32=>29,2=>2,11=>8,21=>17,43=>35,7=>5,10=>7,42=>33…), true, false, false)

In [7]:
mcs.mapping


Dict{Int64,Int64} with 28 entries:
  30 => 27
  1  => 1
  32 => 29
  2  => 2
  11 => 8
  21 => 17
  43 => 35
  7  => 5
  10 => 7
  42 => 33
  29 => 26
  35 => 31
  19 => 15
  17 => 13
  8  => 6
  22 => 21
  6  => 4
  24 => 23
  28 => 25
  5  => 3
  20 => 16
  23 => 22
  14 => 9
  31 => 28
  27 => 24
  ⋮  => ⋮

In [8]:

subg1 = edgesubgraph(mol1, Set(keys(mcs.mapping)))
mol_svg = drawsvg(subg1, 300, 300)
display("image/svg+xml",  mol_svg)